In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
DATA_DIR = r'L:\DCS\Projects\_Legacy\60563434_SWIFT\400_Technical\SWIFT_Workspace\Scenarios\Scenario_2045_S12_Alpha\STM\STM_A\02_TrafficPredictor'
PERF_FILE = os.path.join(DATA_DIR, '04_Results', 'KCP_Performance_NoUpdate_NoPCE_Link53.csv')
LEG_FILE = os.path.join(DATA_DIR, '03_Demand', 'FINAL_Legs_Link_53.csv')

In [3]:
df_perf = pd.read_csv(PERF_FILE)
df_perf.head()

,LINK,DIR,START,END,TTIME,PERSONS,SOV,HOV2,HOV3P,TRUCK,...,AVG_DELAY,AVG_DENSITY,MAX_DENSITY,AVG_QUEUE,MAX_QUEUE,NUM_FAIL,GROUP_VOL,VEH_DIST,VEH_TIME,VEH_DELAY
0,53,0,0:00,0:15,188.2,20.0,0.0,8.0,0.0,2.0,...,0.0,5.1,5.1,0.0,0.0,0.0,0.0,37.771007,0.348861,0.0
1,40709,0,0:00,0:15,81.9,4.0,0.0,2.0,0.0,0.0,...,31.0,2.2,2.2,0.0,0.0,0.0,0.0,1.138218,0.028222,0.0
2,40763,0,0:00,0:15,29.3,2.0,0.0,1.0,0.0,0.0,...,0.0,1.9,1.9,0.0,0.0,0.0,0.0,0.528976,0.008139,0.0
3,41909,0,0:00,0:15,194.7,2.0,0.0,1.0,0.0,0.0,...,147.8,1.2,1.2,0.0,0.0,0.0,0.0,0.203884,0.013028,0.0
4,41943,0,0:00,0:15,48.3,4.0,0.0,2.0,0.0,0.0,...,0.0,2.3,2.3,0.0,0.0,0.0,0.0,1.743557,0.026833,0.0


In [4]:
df_legs = pd.read_csv(LEG_FILE)
df_legs.head()

,HHOLD,LEG,LINK,TIMESTAMP,INTERVAL,TRAVEL_TIME,LENGTH,CUMULATIVE_LENGTH
0,4738,1,53,00:09:27,00:00:00,170.8,20702.0,20702.0
1,4738,2,43026,00:09:43,00:00:00,16.0,1530.0,22232.0
2,4738,3,42800,00:10:20,00:00:00,37.1,3539.0,25771.0
3,4738,4,42915,00:11:00,00:00:00,40.1,3821.0,29592.0
4,4738,5,43241,00:11:34,00:00:00,33.9,3231.0,32823.0


In [5]:
print('Total VMT from Performance File = {:,.0f}'.format(np.sum(df_perf.VEH_DIST)))
print('Total VMT from Plan Legs        = {:,.0f}'.format(np.sum(df_legs.LENGTH)/ 5280.0))

Total VMT from Performance File = 1,425,276
Total VMT from Plan Legs        = 1,428,834


In [6]:
print('Total VHT from Performance File = {:,.0f}'.format(np.sum(df_perf.VEH_TIME)))
print('Total VHT from Plan Legs        = {:,.0f}'.format(np.sum(df_legs.TRAVEL_TIME)/ 3600.0))

Total VHT from Performance File = 38,251
Total VHT from Plan Legs        = 34,104


### Individual Links

### Which Link contributes the most difference?

In [30]:
df_legs_gb = df_legs.groupby(['LINK', 'INTERVAL'], as_index=False).agg({'HHOLD':'count', 'TRAVEL_TIME': 'sum', 'LENGTH': 'sum'})
df_legs_gb['VEH_TIME'] = df_legs_gb['TRAVEL_TIME'] / 3600.0
df_legs_gb['VEH_DIST'] = df_legs_gb['LENGTH'] / 5280.0
df_legs_gb = df_legs_gb.rename(columns={'HHOLD': 'ENTER'})
df_legs_gb.head()

,LINK,INTERVAL,ENTER,TRAVEL_TIME,LENGTH,VEH_TIME,VEH_DIST
0,2,02:00:00,1,41.2,4512.0,0.011444,0.854545
1,2,07:00:00,2,82.2,9024.0,0.022833,1.709091
2,2,07:15:00,1,41.0,4512.0,0.011389,0.854545
3,2,07:30:00,1,41.2,4512.0,0.011444,0.854545
4,2,08:00:00,1,41.1,4512.0,0.011417,0.854545


In [8]:
df_perf['START'] = pd.to_datetime(df_perf.START)
df_perf['INTERVAL'] = pd.DatetimeIndex(df_perf.START).strftime('%H:%M:%S')

In [48]:
df_join = pd.merge(left=df_perf, right=df_legs_gb, 
                   left_on=['LINK', 'INTERVAL'], right_on=['LINK', 'INTERVAL'], suffixes=('_PERF', '_LEG')).loc[:, ['LINK', 'INTERVAL', 'ENTER_LEG', 'ENTER_PERF', 'VEH_TIME_LEG', 'VEH_TIME_PERF', 'VEH_DIST_LEG', 'VEH_DIST_PERF']]
df_join = df_join.sort_values(by=['LINK', 'INTERVAL'])
df_join.head()

,LINK,INTERVAL,ENTER_LEG,ENTER_PERF,VEH_TIME_LEG,VEH_TIME_PERF,VEH_DIST_LEG,VEH_DIST_PERF
2287,2,02:00:00,1,1.0,0.011444,0.011444,0.854545,0.854544
26539,2,07:00:00,2,2.0,0.022833,0.022833,1.709091,1.709088
29105,2,07:15:00,1,1.0,0.011389,0.011389,0.854545,0.854544
31667,2,07:30:00,1,1.0,0.011444,0.011444,0.854545,0.854544
37505,2,08:00:00,1,1.0,0.011417,0.011417,0.854545,0.854544


In [49]:
df_join['VEH_TIME_DIFF'] = df_join.VEH_TIME_LEG - df_join.VEH_TIME_PERF
df_join['VEH_DIST_DIFF'] = df_join.VEH_DIST_LEG - df_join.VEH_DIST_PERF
df_join['ENTER_DIFF'] = df_join.ENTER_LEG - df_join.ENTER_PERF

In [50]:
print('Total VHT   Difference = {:,.0f}'.format(np.sum(df_join.VEH_TIME_DIFF)))
print('Total VMT   Difference = {:,.0f}'.format(np.sum(df_join.VEH_DIST_DIFF)))
print('Total ENTER Difference = {:,.0f}'.format(np.sum(df_join.ENTER_DIFF)))

Total VHT   Difference = -3,767
Total VMT   Difference = 3,977
Total ENTER Difference = 6,831


In [51]:
df_join[['ENTER_DIFF', 'VEH_TIME_DIFF', 'VEH_DIST_DIFF']].describe()

,ENTER_DIFF,VEH_TIME_DIFF,VEH_DIST_DIFF
count,267087.000000,2.670870e+05,2.670870e+05
mean,0.025576,-1.410463e-02,1.488910e-02
std,1.440861,7.668525e-01,1.061487e+00
min,-96.000000,-1.853685e+02,-1.029637e+02
25%,0.000000,-3.333333e-07,6.060606e-08
50%,0.000000,0.000000e+00,6.060606e-07
75%,0.000000,3.333333e-07,2.333333e-06
max,140.000000,5.733746e+01,1.018788e+02


#### Which link has the largest diffference

In [52]:
df_join.loc[df_join.LINK == df_join.loc[df_join.VEH_TIME_DIFF.idxmax()]['LINK']]

,LINK,INTERVAL,ENTER_LEG,ENTER_PERF,VEH_TIME_LEG,VEH_TIME_PERF,VEH_DIST_LEG,VEH_DIST_PERF,VEH_TIME_DIFF,VEH_DIST_DIFF,ENTER_DIFF
22451,24477,06:30:00,1,1.0,0.003306,0.003306,0.0375,0.037500,-4.444444e-07,0.000000,0.0
25157,24477,06:45:00,2,2.0,0.005639,0.005639,0.0750,0.075000,-1.111111e-07,0.000000,0.0
33073,24477,07:30:00,1,1.0,0.163389,0.163389,0.0375,0.037500,-1.111111e-07,0.000000,0.0
39107,24477,08:00:00,37,117.0,5.087944,60.087267,1.3875,2.334694,-5.499932e+01,-0.947194,-80.0
42535,24477,08:15:00,33,94.0,11.602611,113.373923,1.2375,2.579103,-1.017713e+02,-1.341603,-61.0
46026,24477,08:30:00,54,0.0,28.203667,0.088519,2.0250,3.983350,2.811515e+01,-1.958350,54.0
56750,24477,09:15:00,1,1.0,0.001694,0.001694,0.0375,0.037500,4.444444e-07,0.000000,0.0
60297,24477,09:30:00,3,3.0,0.006778,0.006778,0.1125,0.112500,-2.222222e-07,0.000000,0.0
77296,24477,10:45:00,8,8.0,0.019583,0.019583,0.3000,0.299999,3.333333e-07,0.000001,0.0
80735,24477,11:00:00,17,17.0,0.043972,0.043972,0.6375,0.637499,2.222222e-07,0.000001,0.0


In [53]:
df_join.loc[df_join.LINK == df_join.loc[df_join.VEH_TIME_DIFF.idxmax()]['LINK'], ['ENTER_DIFF', 'VEH_TIME_DIFF', 'VEH_DIST_DIFF']].sum()

ENTER_DIFF       -86.000000
VEH_TIME_DIFF   -202.809402
VEH_DIST_DIFF     -4.229944
dtype: float64

In [54]:
# Which households
df_legs.loc[(df_legs.LINK == 24477) & (df_legs.INTERVAL == '16:15:00')].

,HHOLD,LEG,LINK,TIMESTAMP,INTERVAL,TRAVEL_TIME,LENGTH,CUMULATIVE_LENGTH
39887,17784520,81,24477,16:27:42,16:15:00,1534.9,198.0,1.233697e+08
39978,17769164,66,24477,16:26:15,16:15:00,1512.2,198.0,1.236151e+08
42841,18068667,60,24477,16:26:24,16:15:00,1514.2,198.0,1.319147e+08
49854,17570032,81,24477,16:22:26,16:15:00,1460.9,198.0,1.522443e+08
104098,18091050,64,24477,16:27:25,16:15:00,1526.5,198.0,3.152284e+08
104756,17929062,61,24477,16:27:23,16:15:00,1531.6,198.0,3.170205e+08
107538,17907377,61,24477,16:28:22,16:15:00,1532.6,198.0,3.252109e+08
123025,17290928,65,24477,16:16:21,16:15:00,1371.4,198.0,3.695076e+08
170340,17315898,60,24477,16:16:20,16:15:00,1368.7,198.0,5.140445e+08
170719,17417847,66,24477,16:19:00,16:15:00,1406.5,198.0,5.149907e+08


In [56]:
df_legs_gb.loc[(df_legs_gb.LINK == 24477) & (df_legs_gb.INTERVAL == '16:15:00')]

,LINK,INTERVAL,ENTER,TRAVEL_TIME,LENGTH,VEH_TIME,VEH_DIST
139905,24477,16:15:00,140,206657.4,27720.0,57.404833,5.25


In [64]:
df_perf.columns

Index(['LINK', 'DIR', 'START', 'END', 'TTIME', 'PERSONS', 'SOV', 'HOV2',
       'HOV3P', 'TRUCK', 'TRANSIT', 'VOLUME', 'ENTER', 'EXIT', 'FLOW',
       'AVG_SPEED', 'TIME_RATIO', 'AVG_DELAY', 'AVG_DENSITY', 'MAX_DENSITY',
       'AVG_QUEUE', 'MAX_QUEUE', 'NUM_FAIL', 'GROUP_VOL', 'VEH_DIST',
       'VEH_TIME', 'VEH_DELAY', 'INTERVAL'],
      dtype='object')

In [65]:
df_perf.loc[(df_perf.LINK == 24477) & (df_perf.START == '16:15:00'), ['LINK', 'START', 'ENTER', 'SOV', 'HOV2', 'HOV3P', 'TRUCK', 'VOLUME', 'VEH_DIST', 'VEH_TIME']]

,LINK,START,ENTER,SOV,HOV2,HOV3P,TRUCK,VOLUME,VEH_DIST,VEH_TIME
156272,24477,2019-06-17 16:15:00,0.0,0.0,56.0,0.0,138.0,194.0,3.031829,0.067374


In [6]:
df_legs_link_53 = df_legs.loc[df_legs.LINK == 53, :]

In [13]:
df_perf_from_legs_link_53 = df_legs_link_53.groupby('INTERVAL', as_index=False).agg({'HHOLD': 'count', 'TRAVEL_TIME': 'sum', 'LENGTH': 'sum'}).sort_values(by='INTERVAL')
df_perf_from_legs_link_53['VEH_TIME'] = df_perf_from_legs_link_53['TRAVEL_TIME'] / 3600.0
df_perf_from_legs_link_53['VEH_DIST'] = df_perf_from_legs_link_53['LENGTH'] / 5280.0
df_perf_from_legs_link_53

,INTERVAL,HHOLD,TRAVEL_TIME,LENGTH,VEH_TIME,VEH_DIST
0,00:00:00,9,1085.4,186318.0,0.301500,35.287500
1,00:15:00,25,2513.4,517550.0,0.698167,98.020833
2,00:30:00,11,1126.7,227722.0,0.312972,43.129167
3,00:45:00,15,1280.8,310530.0,0.355778,58.812500
4,01:00:00,17,1627.0,351934.0,0.451944,66.654167
5,01:15:00,22,1130.6,455444.0,0.314056,86.258333
6,01:30:00,14,1485.1,289828.0,0.412528,54.891667
7,01:45:00,14,1361.6,289828.0,0.378222,54.891667
8,02:00:00,16,1721.6,331232.0,0.478222,62.733333
9,02:15:00,18,1697.0,372636.0,0.471389,70.575000


In [22]:
df_perf_link_53 = df_perf.loc[df_perf.LINK == 53, ['LINK', 'START', 'ENTER', 'VEH_TIME', 'VEH_DIST']]
df_perf_link_53['START'] = pd.to_datetime(df_perf_link_53.START)
df_perf_link_53['INTERVAL'] = pd.DatetimeIndex(df_perf_link_53.START).strftime('%H:%M:%S')
df_perf_link_53.head()

,LINK,START,ENTER,VEH_TIME,VEH_DIST,INTERVAL
0,53,2019-06-17 00:00:00,10.0,0.348861,37.771007,00:00:00
36,53,2019-06-17 00:15:00,24.0,0.658952,95.537078,00:15:00
121,53,2019-06-17 00:30:00,14.0,0.401944,50.051761,00:30:00
275,53,2019-06-17 00:45:00,15.0,0.385950,56.934878,00:45:00
576,53,2019-06-17 01:00:00,14.0,0.391702,61.608880,01:00:00


In [24]:
df_join = pd.merge(left=df_perf_from_legs_link_53, right=df_perf_link_53, 
                   left_on='INTERVAL', right_on='INTERVAL', suffixes=('_LEG', '_PERF')).loc[:, ['LINK', 'INTERVAL', 'VEH_TIME_LEG', 'VEH_TIME_PERF', 'VEH_DIST_LEG', 'VEH_DIST_PERF']]
df_join.head()

,LINK,INTERVAL,VEH_TIME_LEG,VEH_TIME_PERF,VEH_DIST_LEG,VEH_DIST_PERF
0,53,00:00:00,0.301500,0.348861,35.287500,37.771007
1,53,00:15:00,0.698167,0.658952,98.020833,95.537078
2,53,00:30:00,0.312972,0.401944,43.129167,50.051761
3,53,00:45:00,0.355778,0.385950,58.812500,56.934878
4,53,01:00:00,0.451944,0.391702,66.654167,61.608880


In [26]:
df_join['VEH_TIME_DIFF'] = df_join.VEH_TIME_LEG - df_join.VEH_TIME_PERF
df_join['VEH_DIST_DIFF'] = df_join.VEH_DIST_LEG - df_join.VEH_DIST_PERF

In [27]:
df_join[['VEH_TIME_DIFF', 'VEH_DIST_DIFF']].describe()

,VEH_TIME_DIFF,VEH_DIST_DIFF
count,96.000000,96.000000
mean,-0.696343,0.002709
std,0.777430,28.110182
min,-3.148285,-102.963682
25%,-1.072822,-11.405304
50%,-0.500604,0.895176
75%,-0.131976,13.833896
max,1.500603,101.878849


In [28]:
df_join.iloc[df_join.VEH_DIST_DIFF.idxmax()]

LINK                   53
INTERVAL         06:00:00
VEH_TIME_LEG       13.968
VEH_TIME_PERF     12.4674
VEH_DIST_LEG      1756.53
VEH_DIST_PERF     1654.65
VEH_TIME_DIFF      1.5006
VEH_DIST_DIFF     101.879
Name: 24, dtype: object